## BP Weekly Reporting
### Data Wrangling

Wrangle and repackage weekly data extracts from GA, Facebook and Google AdWords into a tidy dataset prepped for analysis.

In [1]:
# import packages
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from time import strptime
import datetime as dt
import os
import re

In [2]:
## read in FB export 10/22-10/28
Facebook = pd.read_csv('Bluprint_Weekly_Reporting_Template_Facebook.csv')
print(Facebook.info())
Facebook.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 16 columns):
Ad Set Name                 645 non-null object
Day                         645 non-null object
Platform                    645 non-null object
Ad Set ID                   645 non-null float64
Ad Name                     645 non-null object
Ad ID                       645 non-null float64
Impressions                 645 non-null int64
Amount Spent (USD)          645 non-null float64
3-Second Video Views        310 non-null float64
Video Percentage Watched    312 non-null float64
Link Clicks                 515 non-null float64
Website Conversions         448 non-null float64
Conversion Value            645 non-null float64
Video Watches at 95%        306 non-null float64
Reporting Starts            645 non-null object
Reporting Ends              645 non-null object
dtypes: float64(9), int64(1), object(6)
memory usage: 80.7+ KB
None


,Ad Set Name,Day,Platform,Ad Set ID,Ad Name,Ad ID,Impressions,Amount Spent (USD),3-Second Video Views,Video Percentage Watched,Link Clicks,Website Conversions,Conversion Value,Video Watches at 95%,Reporting Starts,Reporting Ends
0,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,10/28/18,instagram,2.384300e+16,Sew_Video_Sew_:15_StartStitching,2.384300e+16,1469,19.85,264.0,18.04,7.0,6.0,0.0,34.0,10/28/18,10/28/18
1,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,10/28/18,instagram,2.384300e+16,Stitch_Video_Knit_:15_StartStitching,2.384300e+16,12648,116.98,2415.0,17.70,27.0,32.0,0.0,285.0,10/28/18,10/28/18
2,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,10/28/18,instagram,2.384300e+16,Dance_Carousel_BalletCarousel,2.384300e+16,1055,6.93,NaN,NaN,NaN,NaN,0.0,NaN,10/28/18,10/28/18
3,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,10/28/18,messenger,2.384300e+16,Dance_Carousel_BalletCarousel,2.384300e+16,6,0.05,NaN,NaN,NaN,NaN,0.0,NaN,10/28/18,10/28/18
4,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,10/28/18,messenger,2.384300e+16,Sew_Photo_FrenchSeam_StartStitching,2.384300e+16,25,0.34,NaN,NaN,1.0,NaN,0.0,NaN,10/28/18,10/28/18


In [4]:
# Drop unneeded columns 
Facebook = Facebook.filter(['Ad Set Name', 'Ad Name', 'Day', 'Platform','Amount Spent (USD)','Impressions','Link Clicks','3-Second Video Views','Video Watches at 95%','Website Conversions'])
Facebook.head()

,Ad Set Name,Ad Name,Day,Platform,Amount Spent (USD),Impressions,Link Clicks,3-Second Video Views,Video Watches at 95%,Website Conversions
0,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Video_Sew_:15_StartStitching,10/28/18,instagram,19.85,1469,7.0,264.0,34.0,6.0
1,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Stitch_Video_Knit_:15_StartStitching,10/28/18,instagram,116.98,12648,27.0,2415.0,285.0,32.0
2,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Dance_Carousel_BalletCarousel,10/28/18,instagram,6.93,1055,NaN,NaN,NaN,NaN
3,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Dance_Carousel_BalletCarousel,10/28/18,messenger,0.05,6,NaN,NaN,NaN,NaN
4,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Photo_FrenchSeam_StartStitching,10/28/18,messenger,0.34,25,1.0,NaN,NaN,NaN


In [5]:
# Change column names 
Facebook.columns = ['Placement', 'Creative', 'Date', 'Platform', 'Cost', 'Impressions', 'Clicks', 'Views', 'Completed Views', 'Conversions']
Facebook.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Video_Sew_:15_StartStitching,10/28/18,instagram,19.85,1469,7.0,264.0,34.0,6.0
1,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Stitch_Video_Knit_:15_StartStitching,10/28/18,instagram,116.98,12648,27.0,2415.0,285.0,32.0
2,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Dance_Carousel_BalletCarousel,10/28/18,instagram,6.93,1055,NaN,NaN,NaN,NaN
3,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Dance_Carousel_BalletCarousel,10/28/18,messenger,0.05,6,NaN,NaN,NaN,NaN
4,Conv_SP_FB_TRIAL_All_All_2Weeksfor1_WCA_Feed_2...,Sew_Photo_FrenchSeam_StartStitching,10/28/18,messenger,0.34,25,1.0,NaN,NaN,NaN


In [6]:
Facebook.Cost.sum()

54054.809999608

In [7]:
# read in Google export
Google = pd.read_csv('Bluprint_Weekly_Reporting_Template_Google.csv')
Google.head()

/Users/christina.caddel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,13,14,15,17,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Bluprint Weekly Reporting Template,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,"October 22, 2018 - October 28, 2018",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ad group,Headline,Headline 1,Headline 2,Expanded Text Ad Headline 3,Short headline,Long headline,Description,Expanded Text Ad Description 2,Description line 1,...,Path 2,Business name,Day,Currency,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/22/18,NaN,$0.00,0,0,0,--,0
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/23/18,NaN,$0.00,0,0,0,--,0
4,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/22/18,NaN,$0.00,0,0,0,--,0


In [8]:
# bring header up from row index 1 and drop imported header and row index 0
Google.drop(Google.index[:1], inplace=True)
new_header = Google.iloc[0]
Google = Google[1:]
Google.columns = new_header
Google = Google.reset_index(drop=True)
Google.head()

1,Ad group,Headline,Headline 1,Headline 2,Expanded Text Ad Headline 3,Short headline,Long headline,Description,Expanded Text Ad Description 2,Description line 1,...,Path 2,Business name,Day,Currency,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/22/18,NaN,$0.00,0,0,0,--,0
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/23/18,NaN,$0.00,0,0,0,--,0
2,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/22/18,NaN,$0.00,0,0,0,--,0
3,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/23/18,NaN,$0.00,0,0,0,--,0
4,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10/24/18,NaN,$0.00,0,0,0,--,0


In [9]:
# filter out unwanted columns
Google = Google.filter(['Ad group','Ad','Day','Cost','Impressions','Clicks','Views','Video played to 100%','Conversions'])
print(Google.info())
Google.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127337 entries, 0 to 127336
Data columns (total 9 columns):
Ad group                127337 non-null object
Ad                      127337 non-null object
Day                     127337 non-null object
Cost                    127337 non-null object
Impressions             127337 non-null object
Clicks                  127337 non-null object
Views                   127337 non-null object
Video played to 100%    127337 non-null object
Conversions             127337 non-null object
dtypes: object(9)
memory usage: 8.7+ MB
None


1,Ad group,Ad,Day,Cost,Impressions,Clicks,Views,Video played to 100%,Conversions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/22/18,$0.00,0,0,0,--,0
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/23/18,$0.00,0,0,0,--,0
2,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/22/18,$0.00,0,0,0,--,0
3,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/23/18,$0.00,0,0,0,--,0
4,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/24/18,$0.00,0,0,0,--,0


In [10]:
# rename columns
Google['Platform'] = 'Google'
Google.columns = ['Placement','Creative','Date','Cost','Impressions','Clicks','Views','Completed Views','Conversions','Platform']
Google.head()

,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,Platform
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/22/18,$0.00,0,0,0,--,0,Google
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/23/18,$0.00,0,0,0,--,0,Google
2,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/22/18,$0.00,0,0,0,--,0,Google
3,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/23/18,$0.00,0,0,0,--,0,Google
4,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/24/18,$0.00,0,0,0,--,0,Google


In [16]:
# create function to strip non-numeric characters from a column
def strp_nonnum(col):
    '''strip string column of non numeric characters, keeping only digits and decimals'''
    col = col.str.extract('([\d]+.?\d*)')
    return col

# strip certain columns of non-numeric characters
cols = ['Cost','Impressions','Clicks','Views','Completed Views','Conversions']
for col in cols:
    Google[col] = strp_nonnum(Google[col])
    Google[col] = Google[col].str.replace(',','').astype(float)

print(Google.info())
Google.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127337 entries, 0 to 127336
Data columns (total 10 columns):
Placement          127337 non-null object
Creative           127337 non-null object
Date               127337 non-null object
Cost               127337 non-null float64
Impressions        127337 non-null float64
Clicks             94569 non-null float64
Views              61801 non-null float64
Completed Views    247 non-null float64
Conversions        32766 non-null float64
Platform           127337 non-null object
dtypes: float64(6), object(4)
memory usage: 9.7+ MB
None


,Placement,Creative,Date,Cost,Impressions,Clicks,Views,Completed Views,Conversions,Platform
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/22/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/23/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
2,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/22/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
3,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/23/18,0.0,0.0,0.0,0.0,NaN,0.0,Google
4,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/24/18,0.0,0.0,0.0,0.0,NaN,0.0,Google


In [17]:
Google.Cost.sum()

62180.17000000001

In [18]:
Google = Google[['Placement','Creative','Date','Platform','Cost','Impressions','Clicks','Views','Completed Views','Conversions']]
Google.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/22/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/23/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
2,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/22/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
3,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/23/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
4,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/24/18,Google,0.0,0.0,0.0,0.0,NaN,0.0


In [14]:
# read in Pinterest export
Pinterest = pd.read_csv('Bluprint_Weekly_Reporting_Template_Pinterest.csv')
Pinterest.head()

,Pin promotion ad group name,Promoted Pin name,Pin image,Promoted Pin ID,Promoted Pin description,Destination URL,Ad's campaign creative type,Spend,Total impressions,Checkout - cost per action,...,Paid average video watch time,Earned MRC video views,Earned video watched at 25%,Earned video watched at 50%,Earned video watched at 75%,Earned video watched at 95%,Earned video watched at 100%,Earned average video watch time,Ad holdout experiment.1,Date
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434910,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,3.232354,429,0.000000,...,0,0,0,0,0,0,0,0,0,10/15/18
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434900,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,62.619226,17269,0.000000,...,0,0,0,0,0,0,0,0,0,10/15/18
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434910,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,20.088206,2551,0.000000,...,0,0,0,0,0,0,0,0,0,10/19/18
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434900,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,1003.948344,326831,0.000000,...,0,0,0,0,0,0,0,0,0,10/19/18
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,P282882420330338688,PP687199434910,Creative classes with experts for $9.99/month....,https://www.mybluprint.com/start?cr_linkid=Con...,REGULAR,80.617012,11609,80.617012,...,0,0,0,0,0,0,0,0,0,10/20/18


In [15]:
# filter out unwanted columns, more efficient than dropping due to quantity of columns
Pinterest = Pinterest.filter(['Pin promotion ad group name','Promoted Pin name','Spend','Total impressions','Total clicks','Total conversions','Total MRC video views','Total video watched at 100%','Date'])
Pinterest.head()

,Pin promotion ad group name,Promoted Pin name,Spend,Total impressions,Total clicks,Total conversions,Total MRC video views,Total video watched at 100%,Date
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,3.232354,429,3,44,0,0,10/15/18
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,62.619226,17269,47,53,0,0,10/15/18
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,20.088206,2551,12,99,0,0,10/19/18
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,1003.948344,326831,656,155,0,0,10/19/18
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,80.617012,11609,42,417,0,0,10/20/18


In [16]:
# rename columns
Pinterest.columns = ['Placement', 'Creative', 'Cost', 'Impressions', 'Clicks', 'Conversions', 'Views', 'Completed Views','Date']
Pinterest.head()

,Placement,Creative,Cost,Impressions,Clicks,Conversions,Views,Completed Views,Date
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,3.232354,429,3,44,0,0,10/15/18
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,62.619226,17269,47,53,0,0,10/15/18
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,20.088206,2551,12,99,0,0,10/19/18
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,1003.948344,326831,656,155,0,0,10/19/18
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,80.617012,11609,42,417,0,0,10/20/18


In [17]:
# add columns to match FB df
Pinterest['Platform'] = 'Pinterest'
Pinterest.head()

,Placement,Creative,Cost,Impressions,Clicks,Conversions,Views,Completed Views,Date,Platform
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,3.232354,429,3,44,0,0,10/15/18,Pinterest
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,62.619226,17269,47,53,0,0,10/15/18,Pinterest
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,20.088206,2551,12,99,0,0,10/19/18,Pinterest
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,1003.948344,326831,656,155,0,0,10/19/18,Pinterest
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,80.617012,11609,42,417,0,0,10/20/18,Pinterest


In [18]:
# reorder dataframe columns to make consistent with Facebook dataframe
Pinterest = Pinterest[['Placement','Creative','Date','Platform','Cost','Impressions','Clicks','Views','Completed Views','Conversions']]
Pinterest.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,10/15/18,Pinterest,3.232354,429,3,0,0,44
1,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,10/15/18,Pinterest,62.619226,17269,47,0,0,53
2,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,10/19/18,Pinterest,20.088206,2551,12,0,0,99
3,Cons_SP_Pin_TRIAL_All_All_Symphony_RT_Feed_201...,All_Static_9.99-MonthlyOffer,10/19/18,Pinterest,1003.948344,326831,656,0,0,155
4,Conv_SP_Pin_TRIAL_All_All_Symphony_WCA_Feed_20...,All_Static_9.99-MonthlyOffer,10/20/18,Pinterest,80.617012,11609,42,0,0,417


In [19]:
# read in Google Analytics export
GoogleAnalytics = pd.read_csv('Bluprint_Weekly_Reporting_Template_Google_Analytics_Raw.csv')
GoogleAnalytics.head()

,# ----------------------------------------,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,# A-01 Bluprint Web Prod - All,NaN,NaN,NaN,NaN,NaN,NaN
1,# LP Visitor Source_Weekly Reporting Pull,NaN,NaN,NaN,NaN,NaN,NaN
2,# 20181022-20181028,NaN,NaN,NaN,NaN,NaN,NaN
3,# ----------------------------------------,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# bring up row index 5 as header and drop row indices 0-4
GoogleAnalytics.drop(GoogleAnalytics.index[:5], inplace=True)
new_header = GoogleAnalytics.iloc[0]
GoogleAnalytics = GoogleAnalytics[1:]
GoogleAnalytics.columns = new_header
GoogleAnalytics = GoogleAnalytics.reset_index(drop=True)
GoogleAnalytics.head()

5,Marketing Action Id,Date,Sessions,New Users,Bounces,Register (Goal 6 Completions),Subscribe (Goal 1 Completions)
0,114592,20181024,"6,862","4,151","5,663",37,1
1,113938,20181022,"6,858","5,667","4,963",230,6
2,113938,20181023,"5,672","4,854","3,815",237,6
3,112180,20181022,"4,229","1,258","1,163",30,51
4,112180,20181023,"4,138","1,206","1,167",39,30


In [21]:
GoogleAnalytics.columns = ['MAID','Date','Sessions','New Users','Bounces','Registrations','Subscriptions']
print(GoogleAnalytics.head())
GoogleAnalytics.info()

     MAID      Date Sessions New Users Bounces Registrations Subscriptions
0  114592  20181024    6,862     4,151   5,663            37             1
1  113938  20181022    6,858     5,667   4,963           230             6
2  113938  20181023    5,672     4,854   3,815           237             6
3  112180  20181022    4,229     1,258   1,163            30            51
4  112180  20181023    4,138     1,206   1,167            39            30
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2929 entries, 0 to 2928
Data columns (total 7 columns):
MAID             2929 non-null object
Date             2929 non-null object
Sessions         2929 non-null object
New Users        2929 non-null object
Bounces          2929 non-null object
Registrations    2929 non-null object
Subscriptions    2929 non-null object
dtypes: object(7)
memory usage: 160.3+ KB


In [22]:
# bring date column to datetime dtype
GoogleAnalytics['Date'] = pd.to_datetime(GoogleAnalytics['Date'], format = '%Y/%m/%d')

# strip non-numeric characters from certain columns
cols = ['Sessions', 'New Users', 'Bounces', 'Registrations', 'Subscriptions']
for col in cols:
    GoogleAnalytics[col] = strp_nonnum(GoogleAnalytics[col])
    GoogleAnalytics[col] = GoogleAnalytics[col].str.replace(',','')
    GoogleAnalytics[col] = pd.to_numeric(GoogleAnalytics[col], errors='coerce')
    
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,114592,2018-10-24,6862,4151,5663,37,1
1,113938,2018-10-22,6858,5667,4963,230,6
2,113938,2018-10-23,5672,4854,3815,237,6
3,112180,2018-10-22,4229,1258,1163,30,51
4,112180,2018-10-23,4138,1206,1167,39,30


In [23]:
# aggregate by MAID and Date, summing remaining columns
GoogleAnalytics = GoogleAnalytics.groupby(['MAID','Date']).sum().reset_index()
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,103706,2018-10-22,1,1,0,0,0
1,103706,2018-10-23,2,2,2,0,0
2,103706,2018-10-24,2,1,1,0,0
3,103706,2018-10-25,4,4,2,0,0
4,103706,2018-10-26,1,0,1,0,0


In [40]:
##MERGE DATA SETS
all_data = pd.concat([Google, Facebook], ignore_index=True)
all_data.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/22/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/23/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
2,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/22/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
3,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/23/18,Google,0.0,0.0,0.0,0.0,NaN,0.0
4,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/24/18,Google,0.0,0.0,0.0,0.0,NaN,0.0


In [41]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127982 entries, 0 to 127981
Data columns (total 10 columns):
Placement          127982 non-null object
Creative           127982 non-null object
Date               127982 non-null object
Platform           127982 non-null object
Cost               127982 non-null float64
Impressions        127982 non-null float64
Clicks             95084 non-null float64
Views              62111 non-null float64
Completed Views    553 non-null float64
Conversions        33214 non-null float64
dtypes: float64(6), object(4)
memory usage: 9.8+ MB


In [42]:
# create new columns and categorize
all_data['MAID'] = all_data['Placement'].str.extract('(_1\d\d\d\d\d)', expand=True)
all_data['MAID'] = all_data['MAID'].str.strip('_')
all_data['Funnel'] = all_data['Placement'].str.split('_').str[0]
#all_data['Tactic'] = all_data['Placement'].str.split('_').str[1]
all_data['Publisher'] = all_data['Placement'].str.split('_').str[2]
all_data['Hook'] = all_data['Placement'].str.split('_').str[3]
all_data['Mega'] = all_data['Placement'].str.split('_').str[4]
all_data['Micro'] = all_data['Placement'].str.split('_').str[5]
all_data['Campaign'] = all_data['Placement'].str.split('_').str[6]
all_data['Targ/aud/pin'] = all_data['Placement'].str.split('_').str[7]
all_data['Keyword'] = all_data['Placement'].str.split('_').str[8]
all_data.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/22/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112081,CONV,GDN,TRIAL,Make,TRIAL,-,RM,Res
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,10/23/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112081,CONV,GDN,TRIAL,Make,TRIAL,-,RM,Res
2,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/22/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112080,CONV,GDN,TRIAL,Savor,TRIAL,-,RM,Res
3,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/23/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112080,CONV,GDN,TRIAL,Savor,TRIAL,-,RM,Res
4,CONV_DIS_GDN_TRIAL_Savor_TRIAL_-_RM_Res_201807...,--,10/24/18,Google,0.0,0.0,0.0,0.0,NaN,0.0,112080,CONV,GDN,TRIAL,Savor,TRIAL,-,RM,Res


In [43]:
all_data.groupby('Mega')['Cost'].nunique()

Mega
All                     743
BP CA                     1
Bluprint LAL             34
Bluprint RM              41
Bluprint RM-Masthead     23
Bluprint SV              88
CI                       25
Compose                 390
Craftsy LAL              89
Craftsy SV               85
Dwell                    55
Engagement                1
KUWK                      1
Lookalike Audience        1
Make                    266
Move                    158
Other                   198
RM                       25
Savor                   210
Shahs of Sunset           6
Site Visitors             1
Stitch                  743
Together                 27
VV                        1
Name: Cost, dtype: int64

In [44]:
all_data['Funnel'].unique()

array(['CONV', 'Conv', 'AW', 'Cons', 'CONS', 'Angela2', 'Angela3',
       'Embroidery1', 'Embroidery3', 'Narwal3', 'Scott2', 'Scott3',
       'Sewing2', 'Sewing4', 'Sewing3', 'Narwal1', 'Angela5',
       'Embroidery2', 'Embroidery4', 'Angela1', 'Angela4', 'Embroidery5',
       'Narwal4', 'Scott5', 'TVC3', 'TVC4', 'TVC1', 'TVC2', 'Narwal2',
       'Scott4', 'Narwal5', 'Scott1', 'Sewing1', 'Angela3 open',
       'Scott3 open', 'Scott5 open', 'Sewing1 open', 'Sewing4 open',
       'Embroidery2 open', 'TVC1 open', 'TVC2 open', 'TVC3 open', 'DIS'],
      dtype=object)

In [45]:
all_data['Publisher'].unique()

array(['GDN', 'G', 'YT', nan, 'GreenLivingEnthusiasts',
       'Do-It-Yourselfers', 'BargainHunters',
       'BluprintRegisteredUsersNew', 'FB'], dtype=object)

In [46]:
all_data = all_data.loc[(all_data['Publisher'] != 'GreenLivingEnthusiasts') & (all_data['Publisher'] != 'Do-It-Yourselfers') & (all_data['Publisher'] != 'BargainHunters') & (all_data['Publisher'] != 'BluprintRegisteredUsersNew')]

In [47]:
all_data['Publisher'].unique()

array(['GDN', 'G', 'YT', nan, 'FB'], dtype=object)

In [48]:
# rename categories for Funnel and Publisher columns
all_data['Funnel'] = all_data['Funnel'].replace({'AW' : 'Awareness', 'Cons' : 'Consideration', 'CONS' : 'Consideration', 'Conv' : 'Conversion', 'CONV' : 'Conversion'})
all_data['Publisher'] = all_data['Publisher'].replace({'G' : 'Google Search', 'GDN' : 'Google Display Network', 'FB' : 'Facebook', 'YT' : 'YouTube'})
all_data['Mega'] = all_data['Mega'].replace({'All':'All/Other','Other':'All/Other','Lookalike Audience':'All/Other','VV':'All/Other','Craftsy LAL':'All/Other','Craftsy SV':'All/Other','Bluprint LAL':'All/Other','Bluprint RM':'All/Other','Bluprint RM-Masthead':'All/Other','Bluprint SV':'All/Other','CI':'All/Other','KUWK':'All/Other','BP CA':'All/Other','Engagement':'All/Other','RM':'All/Other','Shahs of Sunset':'All/Other','Site Visitors':'All/Other'})
print(all_data['Funnel'].unique())
print(all_data['Publisher'].unique())
all_data['Mega'].unique()

['Conversion' 'Awareness' 'Consideration' 'Angela2' 'Angela3'
 'Embroidery1' 'Embroidery3' 'Narwal3' 'Scott2' 'Scott3' 'Sewing2'
 'Sewing4' 'Sewing3' 'Narwal1' 'Angela5' 'Embroidery2' 'Embroidery4'
 'Angela1' 'Angela4' 'Embroidery5' 'Narwal4' 'Scott5' 'TVC3' 'TVC4' 'TVC1'
 'TVC2' 'Narwal2' 'Scott4' 'Narwal5' 'Scott1' 'Sewing1' 'Angela3 open'
 'Scott3 open' 'Scott5 open' 'Sewing1 open' 'Sewing4 open'
 'Embroidery2 open' 'TVC1 open' 'TVC2 open' 'TVC3 open']
['Google Display Network' 'Google Search' 'YouTube' nan 'Facebook']


array(['Make', 'Savor', 'Move', 'All/Other', 'Stitch', 'Dwell', 'Compose',
       'Together', nan], dtype=object)

In [49]:
all_data.loc[(all_data['Funnel'] != 'Awareness') & (all_data['Funnel'] != 'Consideration') & (all_data['Funnel'] != 'Conversion')]

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword
91723,Angela2,Angela2,10/25/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Angela2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91724,Angela2,Angela2,10/28/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Angela2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91725,Angela3,Angela3,10/25/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Angela3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91726,Embroidery1,Embroidery1,10/24/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Embroidery1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91727,Embroidery1,Embroidery1,10/25/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Embroidery1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91728,Embroidery1,Embroidery1,10/26/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Embroidery1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91729,Embroidery1,Embroidery1,10/28/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Embroidery1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91730,Embroidery3,Embroidery3,10/23/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Embroidery3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91731,Embroidery3,Embroidery3,10/26/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Embroidery3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91732,Embroidery3,Embroidery3,10/28/18,Google,0.0,0.0,NaN,NaN,NaN,NaN,NaN,Embroidery3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
all_data = all_data.loc[(all_data['Funnel'] == 'Awareness') | (all_data['Funnel'] == 'Consideration') | (all_data['Funnel'] == 'Conversion')]
all_data.loc[(all_data['Funnel'] != 'Awareness') & (all_data['Funnel'] != 'Consideration') & (all_data['Funnel'] != 'Conversion')]

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,MAID,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword


In [51]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127107 entries, 0 to 127981
Data columns (total 19 columns):
Placement          127107 non-null object
Creative           127107 non-null object
Date               127107 non-null object
Platform           127107 non-null object
Cost               127107 non-null float64
Impressions        127107 non-null float64
Clicks             94664 non-null float64
Views              61691 non-null float64
Completed Views    542 non-null float64
Conversions        33214 non-null float64
MAID               126260 non-null object
Funnel             127107 non-null object
Publisher          127107 non-null object
Hook               127107 non-null object
Mega               127107 non-null object
Micro              127107 non-null object
Campaign           127086 non-null object
Targ/aud/pin       127044 non-null object
Keyword            127044 non-null object
dtypes: float64(6), object(13)
memory usage: 19.4+ MB


In [52]:
Facebook['Platform'].unique()

array(['instagram', 'messenger', 'facebook', 'audience_network'],
      dtype=object)

In [53]:
all_data.loc[all_data['Platform'] == 'instagram', 'Publisher'] = 'Instagram'
all_data['Publisher'].unique()

array(['Google Display Network', 'Google Search', 'YouTube', 'Instagram',
       'Facebook'], dtype=object)

In [54]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127107 entries, 0 to 127981
Data columns (total 19 columns):
Placement          127107 non-null object
Creative           127107 non-null object
Date               127107 non-null object
Platform           127107 non-null object
Cost               127107 non-null float64
Impressions        127107 non-null float64
Clicks             94664 non-null float64
Views              61691 non-null float64
Completed Views    542 non-null float64
Conversions        33214 non-null float64
MAID               126260 non-null object
Funnel             127107 non-null object
Publisher          127107 non-null object
Hook               127107 non-null object
Mega               127107 non-null object
Micro              127107 non-null object
Campaign           127086 non-null object
Targ/aud/pin       127044 non-null object
Keyword            127044 non-null object
dtypes: float64(6), object(13)
memory usage: 19.4+ MB


In [55]:
all_data.loc[all_data['Placement'].str.split('_').str[1] == 'YT', 'Publisher'] = 'YouTube'

In [56]:
# calculate daily impressions by MAID
imp_by_MAID = all_data.groupby(by=['MAID', 'Date'])['Impressions'].sum().reset_index()
imp_by_MAID.head()

,MAID,Date,Impressions
0,111868,10/22/18,126.0
1,111868,10/23/18,126.0
2,111868,10/24/18,118.0
3,111868,10/25/18,139.0
4,111868,10/26/18,145.0


In [57]:
imp_by_MAID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1065 entries, 0 to 1064
Data columns (total 3 columns):
MAID           1065 non-null object
Date           1065 non-null object
Impressions    1065 non-null float64
dtypes: float64(1), object(2)
memory usage: 25.0+ KB


In [58]:
GoogleAnalytics.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions
0,103706,2018-10-22,1,1,0,0,0
1,103706,2018-10-23,2,2,2,0,0
2,103706,2018-10-24,2,1,1,0,0
3,103706,2018-10-25,4,4,2,0,0
4,103706,2018-10-26,1,0,1,0,0


In [59]:
GoogleAnalytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2929 entries, 0 to 2928
Data columns (total 7 columns):
MAID             2929 non-null object
Date             2929 non-null datetime64[ns]
Sessions         2929 non-null int64
New Users        2929 non-null int64
Bounces          2929 non-null int64
Registrations    2929 non-null int64
Subscriptions    2929 non-null int64
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 160.3+ KB


In [60]:
# merge imp_by_MAID and GoogleAnalytics
imp_by_MAID['Date'] = pd.to_datetime(imp_by_MAID['Date'])
GoogleAnalytics = pd.merge(GoogleAnalytics, imp_by_MAID, how='inner', on=['MAID', 'Date'])
print(GoogleAnalytics.head())
GoogleAnalytics.info()

     MAID       Date  Sessions  New Users  Bounces  Registrations  \
0  111868 2018-10-22        73         53       42              3   
1  111868 2018-10-23        79         49       44              3   
2  111868 2018-10-24        70         51       45              3   
3  111868 2018-10-25        60         40       33              2   
4  111868 2018-10-26        57         30       27              0   

   Subscriptions  Impressions  
0              0        126.0  
1              1        126.0  
2              2        118.0  
3              0        139.0  
4              1        145.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 709 entries, 0 to 708
Data columns (total 8 columns):
MAID             709 non-null object
Date             709 non-null datetime64[ns]
Sessions         709 non-null int64
New Users        709 non-null int64
Bounces          709 non-null int64
Registrations    709 non-null int64
Subscriptions    709 non-null int64
Impressions      709 non-nu

In [61]:
##CALCULATE COLUMNS TO BE JOINED INTO OVERALL DATASET
GoogleAnalytics['Session Rate'] = GoogleAnalytics['Sessions'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['New Session Rate'] = GoogleAnalytics['New Users'].div(GoogleAnalytics['Sessions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Bounce Rate'] = GoogleAnalytics['Bounces'].div(GoogleAnalytics['Sessions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Reg Rate'] = GoogleAnalytics['Registrations'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalytics['Sub Rate'] = GoogleAnalytics['Subscriptions'].div(GoogleAnalytics['Impressions'], fill_value=0).replace({np.inf: 0})
GoogleAnalyticsRates = GoogleAnalytics
GoogleAnalyticsRates.head()

,MAID,Date,Sessions,New Users,Bounces,Registrations,Subscriptions,Impressions,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,111868,2018-10-22,73,53,42,3,0,126.0,0.579365,0.726027,0.575342,0.023810,0.000000
1,111868,2018-10-23,79,49,44,3,1,126.0,0.626984,0.620253,0.556962,0.023810,0.007937
2,111868,2018-10-24,70,51,45,3,2,118.0,0.593220,0.728571,0.642857,0.025424,0.016949
3,111868,2018-10-25,60,40,33,2,0,139.0,0.431655,0.666667,0.550000,0.014388,0.000000
4,111868,2018-10-26,57,30,27,0,1,145.0,0.393103,0.526316,0.473684,0.000000,0.006897


In [62]:
GoogleAnalyticsRates.drop(columns=['Sessions', 'New Users', 'Bounces', 'Registrations', 'Subscriptions', 'Impressions'], axis=1, inplace=True)
GoogleAnalyticsRates.head()

,MAID,Date,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,111868,2018-10-22,0.579365,0.726027,0.575342,0.023810,0.000000
1,111868,2018-10-23,0.626984,0.620253,0.556962,0.023810,0.007937
2,111868,2018-10-24,0.593220,0.728571,0.642857,0.025424,0.016949
3,111868,2018-10-25,0.431655,0.666667,0.550000,0.014388,0.000000
4,111868,2018-10-26,0.393103,0.526316,0.473684,0.000000,0.006897


In [63]:
##JOIN NEW GOOGLE ANALYTICS TO ALL_DATA_MERGE
all_data['Date'] = pd.to_datetime(all_data['Date'])
final_data_merge = pd.merge(all_data, GoogleAnalyticsRates, how='inner', on=['MAID', 'Date'])
print(final_data_merge.info())
final_data_merge.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109850 entries, 0 to 109849
Data columns (total 24 columns):
Placement           109850 non-null object
Creative            109850 non-null object
Date                109850 non-null datetime64[ns]
Platform            109850 non-null object
Cost                109850 non-null float64
Impressions         109850 non-null float64
Clicks              82966 non-null float64
Views               51765 non-null float64
Completed Views     532 non-null float64
Conversions         30134 non-null float64
MAID                109850 non-null object
Funnel              109850 non-null object
Publisher           109850 non-null object
Hook                109850 non-null object
Mega                109850 non-null object
Micro               109850 non-null object
Campaign            109842 non-null object
Targ/aud/pin        109842 non-null object
Keyword             109842 non-null object
Session Rate        109850 non-null float64
New Session Rate    

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,...,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Session Rate,New Session Rate,Bounce Rate,Reg Rate,Sub Rate
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.103448,0.827586,NaN,NaN
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.103448,0.827586,NaN,NaN
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.103448,0.827586,NaN,NaN
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.103448,0.827586,NaN,NaN
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_BPRM_Res_20180...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,BPRM,Res,0.0,0.103448,0.827586,NaN,NaN


In [64]:
# create new columns by multiplying others
final_data_merge['Sessions'] = final_data_merge['Impressions'].mul(final_data_merge['Session Rate'])
final_data_merge['New Sessions'] = final_data_merge['Sessions'].mul(final_data_merge['New Session Rate'])
final_data_merge['Bounces'] = final_data_merge['Sessions'].mul(final_data_merge['Bounce Rate'])
final_data_merge['Registrations'] = final_data_merge['Impressions'].mul(final_data_merge['Reg Rate'])
final_data_merge['Subscriptions'] = final_data_merge['Impressions'].mul(final_data_merge['Sub Rate'])
final_data_merge.drop(columns=['Session Rate', 'New Session Rate', 'Bounce Rate', 'Reg Rate', 'Sub Rate'], axis=1, inplace=True)
final_data_merge.head()

,Placement,Creative,Date,Platform,Cost,Impressions,Clicks,Views,Completed Views,Conversions,...,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.0,0.0,NaN,NaN
1,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.0,0.0,NaN,NaN
2,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.0,0.0,NaN,NaN
3,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_RM_Res_2018071...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,RM,Res,0.0,0.0,0.0,NaN,NaN
4,CONV_DIS_GDN_TRIAL_Make_TRIAL_-_BPRM_Res_20180...,--,2018-10-22,Google,0.0,0.0,0.0,0.0,NaN,0.0,...,Make,TRIAL,-,BPRM,Res,0.0,0.0,0.0,NaN,NaN


In [65]:
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109850 entries, 0 to 109849
Data columns (total 24 columns):
Placement          109850 non-null object
Creative           109850 non-null object
Date               109850 non-null datetime64[ns]
Platform           109850 non-null object
Cost               109850 non-null float64
Impressions        109850 non-null float64
Clicks             82966 non-null float64
Views              51765 non-null float64
Completed Views    532 non-null float64
Conversions        30134 non-null float64
MAID               109850 non-null object
Funnel             109850 non-null object
Publisher          109850 non-null object
Hook               109850 non-null object
Mega               109850 non-null object
Micro              109850 non-null object
Campaign           109842 non-null object
Targ/aud/pin       109842 non-null object
Keyword            109842 non-null object
Sessions           109850 non-null float64
New Sessions       109850 non-null float

In [66]:
##ADD REMAINING COLUMNS
final_data_merge['Start Date'] = final_data_merge.groupby(['Placement'])['Date'].transform(min)
final_data_merge['Video v Static'] = np.where(final_data_merge['Views']>=1, 'Video', 'Static')
final_data_merge['Impressions with Video View'] = np.where(final_data_merge['Views']>=1, final_data_merge['Impressions'], 0)
final_data_merge['Reporting Month'] = final_data_merge['Date'].dt.strftime('%B')
final_data_merge['Week Start'] = final_data_merge['Date'].dt.to_period('W').apply(lambda r: r.start_time)
final_data_merge['Reporting Period'] = final_data_merge['Date'].dt.to_period('W').apply(lambda r: r.start_time)

In [67]:
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109850 entries, 0 to 109849
Data columns (total 30 columns):
Placement                      109850 non-null object
Creative                       109850 non-null object
Date                           109850 non-null datetime64[ns]
Platform                       109850 non-null object
Cost                           109850 non-null float64
Impressions                    109850 non-null float64
Clicks                         82966 non-null float64
Views                          51765 non-null float64
Completed Views                532 non-null float64
Conversions                    30134 non-null float64
MAID                           109850 non-null object
Funnel                         109850 non-null object
Publisher                      109850 non-null object
Hook                           109850 non-null object
Mega                           109850 non-null object
Micro                          109850 non-null object
Campaign         

In [68]:
#REORGANIZE REMAINING COLUMNS
final_data_merge = final_data_merge[['Funnel', 'Publisher', 'Hook', 'Mega', 'Micro', 
                                     'Campaign', 'Targ/aud/pin', 'Keyword', 'Start Date', 'MAID', 
                                     'Video v Static', 'Impressions with Video View', 'Week Start', 
                                     'Reporting Month', 'Reporting Period', 'Placement', 'Creative', 'Date', 
                                     'Cost', 'Impressions', 'Clicks', 'Views', 'Completed Views', 'Conversions', 
                                     'Sessions', 'New Sessions', 'Bounces', 'Registrations', 'Subscriptions']]
final_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109850 entries, 0 to 109849
Data columns (total 29 columns):
Funnel                         109850 non-null object
Publisher                      109850 non-null object
Hook                           109850 non-null object
Mega                           109850 non-null object
Micro                          109850 non-null object
Campaign                       109842 non-null object
Targ/aud/pin                   109842 non-null object
Keyword                        109842 non-null object
Start Date                     109850 non-null datetime64[ns]
MAID                           109850 non-null object
Video v Static                 109850 non-null object
Impressions with Video View    109850 non-null float64
Week Start                     109850 non-null datetime64[ns]
Reporting Month                109850 non-null object
Reporting Period               109850 non-null datetime64[ns]
Placement                      109850 non-null object


In [69]:
final_data_merge['Date'].unique()

array(['2018-10-22T00:00:00.000000000', '2018-10-23T00:00:00.000000000',
       '2018-10-24T00:00:00.000000000', '2018-10-25T00:00:00.000000000',
       '2018-10-26T00:00:00.000000000', '2018-10-27T00:00:00.000000000',
       '2018-10-28T00:00:00.000000000'], dtype='datetime64[ns]')

In [70]:
final_data_merge['Targ/aud/pin'].unique()

array(['RM', 'BPRM', 'EG', 'JessicaLong', 'Celebrity', 'Generic',
       'Celebrrity', 'FabioViviani', 'E', 'Accessories', 'Doodle-Wars',
       'JoshuaVergara', 'BrookeLark', 'JoshuaJohnRussell', 'BMM', 'CA',
       'AA', 'IM', 'SV', 'LAL', 'SV LAL', 'vv', 'VV', 'im', 'aa', 'CI',
       nan, 'WCA', 'RT', 'Interest'], dtype=object)

In [71]:
final_data_merge.Cost.sum()

112371.78999960801

In [76]:
cols = ['Cost','Impressions','Clicks','Views','Completed Views','Conversions','Sessions','New Sessions','Bounces','Registrations','Subscriptions']
for col in cols:
    final_data_merge[col].fillna(0)

In [77]:
final_data_merge.head()

,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Start Date,MAID,...,Impressions,Clicks,Views,Completed Views,Conversions,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
1,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
2,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
3,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
4,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,BPRM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN


In [73]:
final_data_merge.to_csv('weekly_data_merge.csv')

### To Do


✅ drop 'Tactic' column 

✅ rename 'Video Completion' to 'Completed Views'

- audit columns for:
    - data type
    - quality
    - etc.
    
✅ Refine 'Mega' column to only include unique values, not duplicates with slight variations OR assign "All/Other" properly

In [74]:
final_data_merge.head()

,Funnel,Publisher,Hook,Mega,Micro,Campaign,Targ/aud/pin,Keyword,Start Date,MAID,...,Impressions,Clicks,Views,Completed Views,Conversions,Sessions,New Sessions,Bounces,Registrations,Subscriptions
0,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
1,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
2,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
3,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,RM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
4,Conversion,Google Display Network,TRIAL,Make,TRIAL,-,BPRM,Res,2018-10-22,112081,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN
